In [1]:
! pip install -U langchain_community tiktoken langchain-cohere langchainhub chromadb langchain langgraph  tavily-python langchain_groq langchain_google_genai --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 667.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 14.2 MB/s eta 0:00

In [2]:
import os
from google.colab import userdata
groq_api_key = userdata.get("GROQ_API_KEY")

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key =userdata.get("LANGSMITH_API_KEY")

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
print("model retrieved")

model retrieved


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
class domainrouter(BaseModel):
    proj_domain: str = Field(
        description="answer with the domain in which the project lies in , whether in 'webapp' , 'blockchain' , 'mobile' , 'aiml' , 'backend'. Return only the domain name only "
    )

structured_llm_grader = llm.with_structured_output(domainrouter)
system = """You are a professional software developer who has an expertise in the web development ,backend ,  blockchain , aiml and mobile development role ,
    you need to classify whether the project details given by the user lies in which domain , whether it's related to website development , in this case return 'webapp' ,or the project is of backend then return 'backend' or the project is for the blockchain domain and in this case return 'blockchain'
    whether the project is related to AIML then return with the 'aiml' domain , or else if the project contains app development then return with the 'mobile'.
    remember that this decision is very crucial dont give any kind of wrong answer. If the website making or app making is required in aiml or blockchain domain , then categorize them into 'webapp' or 'mobile' only """
route_domain = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} "),
    ]
)
domain_router = route_domain | structured_llm_grader
query = "a tic tac toe website "
proj_domain = domain_router.invoke({"query": query})
domain = proj_domain.proj_domain



In [7]:
print(domain)

webapp


In [8]:
from re import template
class templaterouter(BaseModel):
    structure_template: str = Field(
        description="decide which template is to be used to make the skeleton codebase of the project , the project domain will be provided. If the domain is 'webapp' you need to classify the template into 'angular', 'NextJS' , 'Astro' , 'REACT_Javascript' , 'REACT_typescript' , 'Simple' , 'Svelte' . If the project domain is Backend development then classify the project into 'Go' , 'Flask' , 'Node' , 'Rust' , 'Laravel' , 'Django' . If the domain is in 'mobile' then classify the project into these templates 'Flutter' , 'react-native' . If the project domain is 'aiml' then classify the project with 'Langchain' . if the domain is 'blockchain' then just simply return 'nil'"
    )

structured_llm_grader = llm.with_structured_output(templaterouter)
system = """You are a professional software developer who has an expertise in the web development , blockchain , aiml and mobile development role ,
    you are extremely specialised in creating the codebases of the complex projects and know which pre existing template to use for various projects , We currently have these templates in the 'webapp' domain which are
     'angular', 'NextJS' , 'Astro' , 'REACT_Javascript' , 'REACT_typescript' , 'SimpleHTML' , 'Svelte' . In the 'backend' domain we have 'Go' , 'Flask' , 'Node' , 'Rust' , 'Laravel' , 'Django' templates . In the 'mobile' domain in which we have to app development we have 'Flutter' , 'react-native' templates . If the domain is AIML , then we have 'langchain' as the domain
     Carefully return me with the project template only on the basis of the domain and the project details. """
template_domain = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} , domain : {domain}"),
    ])
template_router = template_domain | structured_llm_grader
template = template_router.invoke({"query": query , "domain": domain})
template_structure = template.structure_template
print(template_structure)


SimpleHTML


In [9]:
class queryreframer(BaseModel):
    new_query: str = Field(
        description = "You a question re-writer that converts an input question to a better version that is optimized and does not looses any of he context "
    )
structured_llm_grader = llm.with_structured_output(queryreframer)
system = """
You are a professional at reframing the user inputs without loosing the context and the meaning of the query , make changes in the query such that its relevant to the coding project making , make the query such that its relevant to a model which makes projects on the basis of the user queries . Dont make it more complex than it was , and dont add any kind of new functionalities
list out the functionalities as well of the project"""
query_reframing = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "query : {query}"),
    ]
)
query_reframer = query_reframing | structured_llm_grader
print(query_reframer.invoke({"query": query}))

new_query='Design a web-based Tic Tac Toe game with the following functionalities: user authentication, game board display, move validation, win/loss detection, and score tracking. The project should allow users to play against each other or against the computer, and provide options for restarting the game or viewing game history.'


In [10]:
class templatechecker(BaseModel):
    template_compatibility : str = Field(
        description = "Check whether the given user query can be made with the template given , return 'yes' or 'no' . "
    )
structured_llm_grader = llm.with_structured_output(templatechecker)
system = """You are a professional software developer , you are expert at checking whether the project query given by the user can be made from the template , Return 'yes' if the template is correct and 'no' if the template is incorrect"""
template_checking = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "query : {query} , template : {template}"),
    ]
)
template_checker = template_checking | structured_llm_grader
print(template_checker.invoke({"query": query , "template": template_structure}))

template_compatibility='yes'


In [11]:
class directorynamer(BaseModel):
    proj_name : str = Field(
        description = "Give the appropriate name for the project"
    )
structured_llm_grader = llm.with_structured_output(directorynamer)
system = """
You are a professional software developer and are extremely professional at giving names to the projects . You have to name the project based on the domain , and the project details """
directory_naming = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} , domain : {domain}"),
    ]
)
directory_namer = directory_naming | structured_llm_grader
print(directory_namer.invoke({"query": query , "domain": domain}))

proj_name='TicTacToeWebApp'
